In [1]:
import pandas as pd
import numpy as np
import collections
import json
import re
from tqdm import tqdm
import pickle

tqdm.pandas()

In [3]:
root = "../../../data/interim/"

df_bert = pd.read_csv(root + "DF_GoogleForms_BERT_AllData.csv", 
                        index_col=0)
df_jacc = pd.read_csv(root + "DF_GoogleForms_Jaccard_AllData.csv",
                        index_col=0)

df_jacc["SIM"] = "Jacc"
df_bert["SIM"] = "Bert"

In [4]:
root = "../../../data/processed/"

f = open(F"{root}Sentences_Pierre.pkl", 'rb')
sentences_Pierre = pickle.load(f)
species_Pierre =  sentences_Pierre.keys()

f = open(F"{root}Sentences_Andrei.pkl", 'rb')
sentences_Andrei = pickle.load(f)
species_Andrei = sentences_Andrei.keys() 

f = open(F"{root}Sentences_Kissling.pkl", 'rb')
sentences_Kissling = pickle.load(f)
species_Kissling = sentences_Kissling.keys() 


In [5]:
def datalist(species):

    name = ""
    if species in species_Pierre:
        name = "PlantNet"
    elif species in species_Andrei:
        name = "Caribbean"
    elif species in species_Kissling:
        name = "Palm"

    return name

def ascii_converter(text):

    if type(text) != str:
        return text

    patterns = [
        (r"â\x80\x93", " to "),
        (r"\xa0", " "),
        (r"Â", ""),
        (r"\s,", ","),  
        (r"\s+", " "),
        (r"\s:", ":"),
        (r"\sÃ\x97", ","),
        (r"â\x80\x94", " ")
    ]
    for (pattern, replacement) in patterns:
        text = re.sub(pattern, replacement, text)

    text = text.lstrip(".")
    
    return text

In [10]:
df = pd.concat(
    [df_bert, df_jacc]
)

# Datalist species
df["Dataset"] = df["Species"].apply(datalist)

# Note to self: Could be done using Pandas
columns = [
    "1", "2", "3", "4", "5"
]

# Regexes
for column in columns:
    df.loc[:, column] = df[column].apply(ascii_converter)

In [13]:
df_Caribbean = df[df["Dataset"] == "Caribbean"]
JACC_Thresh = df_Caribbean.describe().loc["75%", "Jaccard Sum"]
BERT_Thresh = df_Caribbean.describe().loc["75%", "Sim Sum"]
df_Caribbean_bert_sample = df_bert[df_bert["Sim Sum"] > BERT_Thresh]
df_Caribbean_Jacc_sample = df_jacc[df_jacc["Jaccard Sum"] > JACC_Thresh]
df_Caribbean_75 = pd.concat(
    [df_Caribbean_bert_sample, df_Caribbean_Jacc_sample]
)

folder = "../../../data/processed/"
for i in range(1, 4):
    df_sample = df_Caribbean_75.sample(n=20, 
                            random_state=333 + i, 
                            replace=False)

    df_sample.to_csv(f"{folder}top_sents_all_AllSentencesAgainstTrait_Random20Subset_Caribbean_{i}.csv", sep="\t")


# df_Palm = df[df["Dataset"] == "Palm"]
# df_PlantNet = df[df["Dataset"] == "PlantNet"]


# JACC_Thresh = df_jacc.describe().loc["75%", "Jaccard Sum"]
# BERT_Thresh = df_bert.describe().loc["75%", "Sim Sum"]

# df_bert_sample = df_bert[df_bert["Sim Sum"] > BERT_Thresh]\
#     # .sample(n=20, replace=False, random_state=333)

# df_jacc_sample = df_jacc[df_jacc["Jaccard Sum"] > JACC_Thresh]\
#     # .sample(n=20, replace=False, random_state=333)

In [ ]:
JACC_Thresh = df_jacc.describe().loc["75%", "Jaccard Sum"]
BERT_Thresh = df_bert.describe().loc["75%", "Sim Sum"]

df_bert_sample = df_bert[df_bert["Sim Sum"] > BERT_Thresh]\
    # .sample(n=20, replace=False, random_state=333)

df_jacc_sample = df_jacc[df_jacc["Jaccard Sum"] > JACC_Thresh]\
    # .sample(n=20, replace=False, random_state=333)

In [ ]:
df = pd.concat(
    [df_bert_sample, df_jacc_sample]
)

In [ ]:
folder = "../../../data/processed/"

df.sample(n=20, random_state=123, replace=False)\
    .to_csv(f"{folder}top_sents_all_AllSentencesAgainstTrait_Random20Subset_demo.csv", sep="\t")

for i in range(1, 11):
    df_sample = df.sample(n=20, 
                        random_state=333 + i, 
                        replace=False)

    df_sample.to_csv(f"{folder}top_sents_all_AllSentencesAgainstTrait_Random20Subset_{i}.csv", sep="\t")

In [ ]:
df.sample(n=20, random_state=123, replace=False)

In [ ]:
df.sample(n=20, random_state=123, replace=False)\
    .loc[11007, "2"]